In [1]:
import torch

x = torch.FloatTensor([[1, 2],
                       [3, 4],
                       [5, 6]])
y = torch.FloatTensor([[1, 2],
                       [1, 2]])

print(x.size(),y.size())

print(x + y)

torch.Size([3, 2]) torch.Size([2, 2])


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 0

In [ ]:
x = torch.FloatTensor(
    [[[1, 2],
     [3, 4],
     [5, 6]],

    [[7, 8],
     [9, 10],
     [11, 12]],

    [[13, 14],
     [15, 16],
     [17, 18]]] # (3, 3, 2)
)
y = torch.FloatTensor(
    [[[1, 2, 2],
     [1, 2, 2]],

    [[1, 3, 3],
     [1, 3, 3]],

    [[1, 4, 4],
     [1, 4, 4]]] # (3, 2, 3)
)

print(x.size(), y.size())


torch.Size([3, 3, 2]) torch.Size([3, 2, 3])


In [3]:
z = torch.matmul(x, y)  # 행렬 곱 수행 (마지막 2차원 기준)
print(z.size())

torch.Size([3, 3, 3])


- 마지막 2개 차원을 기준으로 행렬곱 수행
- batch 차원은 같아야 한다. (아니면 11이라도 되서 브로드캐스팅이라도 되어야 한다.)   
(3, 3, 2) x (3, 2, 3) = (3, 3, 3)

In [6]:
# 3차원에서만 사용할 수 있는 행렬곱 연산 bmm(batch matrix multiplication)
z = torch.bmm(x, y) # 행렬 곱 수행 (마지막 2차원 기준)
print(z.size())

torch.Size([3, 3, 3])


torch.bmm
입력이 반드시 3차원 텐서 (B, N, M) @ (B, M, P) = (B, N, P)   
각 batch는 batch마다 독립적으로 행렬곱 연산   
| 구분 | torch.matmul | torch.bmm |
| --- | --- | --- |
| 입력 차원 | 1D ~~ ND | 3D만 |
| 출력 차원 | 자동 해석 + 브로드캐스팅 | Batch크기 동일해야 한다. |
| 연산 범위 | 범용 행렬곱 | 배치 행렬곱 |
| 유연성 | 높다 | 낮다 |

bmm은 3차원 배치 행렬곱 전용   
matmul은 차원 수에 따라 자동으로 처리되는 범용 행렬곱

.T(전치): 를 이용해서 입력 차원과 곱셈이 가능하도록 맞춘다.

In [8]:
x = torch.tensor(
    [[1., 2., 3.]]
)
W = torch.tensor(
    [[0.1, 0.2, 0.3],
     [0.4, 0.5, 0.6]]
)

print(x.size(), W.size()) # (1, 3), (2, 3)

torch.Size([1, 3]) torch.Size([2, 3])


In [11]:
# y = torch.matmul(x, W)

In [10]:
W.T.size()

torch.Size([3, 2])

.T (전치 Transpose) : (2, 3) -> (3, 2)

In [12]:
y = torch.matmul(x, W.T) # (1, 3) @ (3, 2) -> (1, 2)

y.size()

torch.Size([1, 2])

행렬곱 절대원칙   
행렬곱에서 마지막 두 차원은 반ㄷ드시 아래와 같아야 한다.   
(..., N, M) @ (... M, P) -> M == M

In [ ]:
x = torch.FloatTensor(
    [[[1, 2],
     [3, 4],
     [5, 6]],

    [[7, 8],
     [9, 10],
     [11, 12]],

    [[13, 14],
     [15, 16],
     [17, 18]]] # (3, 3, 2)
)
W = torch.tensor(
    [[0.1, 0.2, 0.3],
     [0.4, 0.5, 0.6]]
)

print(x.size(), W.size())


torch.Size([3, 3, 2]) torch.Size([2, 3])


In [14]:
z = torch.matmul(x, W)
print(z.size())

torch.Size([3, 3, 3])


브로드캐스팅 (2, 3) --> batch 차원 추가 (11, 2, 3) -> batch 방향으로 복제 -> (3, 2, 3)   
(3, 3 ,2) @ (3, 2, 3) => (3, 3, 3)

In [15]:
x = torch.FloatTensor([[[1, 2],
                        [3, 4],
                        [5, 6]],

                        [[7, 8],
                        [9, 10],
                        [11, 12]],

                        [[13, 14],
                        [15, 16],
                        [17, 18]]]) # (3, 3, 2)

W = torch.tensor([[[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]]]) # (4, 2, 3)

print(x.size(),W.size())

torch.Size([3, 3, 2]) torch.Size([4, 2, 3])


batch 차원이 3 !== 4 (맞지 않음)   
내적 계산은 맞으나 자동으로 batch가 브로드캐스팅 되지않음.

- 내적 차원 불일치 한 경우 : 불가
- batch 차원이 맞지 않는데 브로드캐스팅 조건에도 맞지 않은 경우 : 불가
- 내적 차원 일치 + batch 차원이 맞는 경우 : 가능
- 내적 차원 일치 + batch 차원이 맞지 않는데, batch차원 브로드캐스팅 가능한 경우 : 가능 (자동 처리)   
=> batch는 행렬곱 연산 결과 자체에는 상관없으나, 형태가 맞아야 한다.

In [ ]:
x = torch.randn(2, 3, 2)
W = torch.randn(2, 4, 2)

# y = torch.bmm(x, W.T) -> 3차원 이상은 transpose()를 이용하여 명시적으로 교환
y = torch.bmm(x, W.transpose(1, 2))
y.size()

torch.Size([2, 3, 4])

transpose(dim1, dim2) : 지정된 2개의 차원을 교환   
(2, 4, 2) -> (2, 2, 4) : 내적 차원 계산할 수 있도록 전치

In [20]:
import torch
import torch.nn as nn

x = torch.tensor(
    [[1., 2., 3.]]
)

W = torch.tensor(
    [[0.1, 0.2, 0.3],
     [0.4, 0.5, 0.6]]
)

print(x.size()) # (1, 3)
print(W.size()) # (2 ,3)

torch.Size([1, 3])
torch.Size([2, 3])


nn.Linear 레이어 생성 (bias 생략) : 수동으로 만든 행렬곱과 정확히 동일한 결과가 나온다.

In [21]:
linear = nn.Linear(
    in_features = 3,  # 입력 벡터의 길이(x의 열), 행렬곱에서는 입력의 열 차원
    out_features = 2, # 출력 벡터의 길이(출력의 행)
    bias = False      # bias 항 제외 -> 순수 행렬곱만 수행한 결과
)


In [22]:
# W를 nn.Linear 레이어에 직접 복사
with torch.no_grad():      # 그래디언트 추적 비활성화 (가중치 수동 조절)
    linear.weight.copy_(W) # Linear 레이어의 가중치를 W값으로 직접 덮어쓰기


copy_(W) : 대입해주는게 아니라 값만 복사   
=> 파라미터 객체는 그대로 유지

In [23]:
y2 = linear(x)
print(y2, y2.size())

tensor([[1.4000, 3.2000]], grad_fn=<MmBackward0>) torch.Size([1, 2])


linear(x) 연산을 했는데 내부적으로 자동으로 x @ W.T 시켜서 계산

In [25]:
# 항등함수
import numpy as np

def identity_function(x):
    return x

x = np.array([1.0, 2.0, 3.0])
print(identity_function(x))

[1. 2. 3.]


이진분류 (임계값 기준)   
출력값 >= 0.5 : 클래스 1   
출력값 < 0.5 : 클래스 0   
   
   다중분류 (서로 독립된 확률) -> 하나의 데이터가 여러 클래스에 동시에 속할 수 있는 문제를 해결할 수 있다.   
      
입력값이 한없이 0이나 1에 가까워지면 기울기가 0에 수렴하는 (기울기 소실) 문제가 발생할 수 있다.   
=> 주로 은닉층에서는 사용하지 않고 출력층에서만 주로 사용한다.   
   
   손실함수로는 BCE 사용한다

$$
\frac{e^{x_i}}{\sum_j e^{x_j}}
=
\frac{e^{x_i - c}}{\sum_j e^{x_j - c}}
$$